In [1]:
#this is a roadmap from beginning truth to the final iteration of iPto19

In [2]:
import cobra
from cobra import Model, Gene, Reaction, Metabolite
from cobra.core.gene import parse_gpr,eval_gpr
import pandas as pd
import numpy
import medusa
from medusa.test import load_universal_modelseed
import csv
import matplotlib.pylab as plt
from pickle import load

V1 was originally created from the feeder document, which was derived from looking at reactions present in the draft reconstruction and confirming the activity and presence of an enzyme. This is the document that starts the whole process. It was converted first into an sbml file format to be later read as a cobra model object. 

In [3]:
#these are the universal variables 
seed_draft = cobra.io.load_json_model('../data/modelseed_data/modelseed_draft_psy_DC3000.json')
universal = load_universal_modelseed()
filepath_model = "../results/reconstructions/"
filesuff_xml = ".xml"
filesuff_csv = ".csv"
pa = cobra.io.read_sbml_model('../data/previous_reconstructions/iPAE1146_resaved.xml')

'' is not a valid SBML 'SId'.


In [4]:
#this is a general excel script that is used to convert each of the excel files into a json model, which is then converted into an sbml model for ease of access
#generate the list of reactions to be used for generating the model, from the curation file
pst_feeder_xl_file = pd.read_excel('../data/PST_feeder.xlsx')

filevar = "pst_feeder"

#at this point we have a grab bag of reactions ready to be inserted into the model, which we have yet to create
model = Model('PST') #empty model

#add reactions from the grab bag
for i in range(0,len(pst_feeder_xl_file)):
    reaction_row = pst_feeder_xl_file.iloc[i,]
    reaction_id = str(reaction_row['Reaction ID'])[:-1]
    reaction_to_add = universal.reactions.get_by_id(reaction_id).copy()
    # add subsystem for the reaction
    reaction_to_add.subsystem = reaction_row ['Subsystems']
    # add gpr for the reaction
    reaction_to_add.gene_reaction_rule = reaction_row['GPR'] 
    # set the bounds based on the manual curation we did (rxns in universal are reversible)
    # calling .item() is necessary to convert the numpy datatype into a native Python float
    # to be compatible with the libsbml parser/writer.
    reaction_to_add.bounds = (reaction_row['Lower bound'].item(),
                              reaction_row['Upper bound'].item())
    model.add_reaction(reaction_to_add)

save_dir_model = filepath_model+filevar+filesuff_xml
write_dir = filepath_model + filevar + filesuff_csv

with open(write_dir, 'w', newline='') as csvfile:
    csvfile = csv.writer(csvfile, delimiter=',')
    for rxn in model.reactions:
        csvfile.writerow([rxn.id, rxn.name, rxn.reaction, rxn.gene_reaction_rule,rxn.lower_bound, rxn.upper_bound])

cobra.io.write_sbml_model(model, save_dir_model)
model

Ignoring reaction 'rxn00182_c' since it already exists.
Ignoring reaction 'rxn00260_c' since it already exists.
Ignoring reaction 'rxn00493_c' since it already exists.
Ignoring reaction 'rxn00527_c' since it already exists.
Ignoring reaction 'rxn01640_c' since it already exists.


Name,PST
Memory address,0x01230b6860
Number of metabolites,242
Number of reactions,149
Number of groups,0
Objective expression,0
Compartments,"c, e"


V1 was not integrated with the orthologs between PAO1 and Pst yet. So, there are a few scripts and outputs that achive this. Originally, this was done through the scripts "ortholog_compiler", "comp_list", "rxns_left", "rxn_list_comp" all feeding into the newly updated PST_curation.xlsx. I will combine all the scripts here in the next cells.

In [5]:
filevar = "PSTv1"

#this is the script originally called "ortholog_to_list.py" which was used to compare the PAO1 and PST orthologs to the list of PAO1 gprs in PAO1 reactions
#followed by comparing the list of PAO1 reactions to the list of reactions left to be curated in PST draft. If there is an ortholog between PAO1 and PSt, and it has a
#matching reaction that is defined in PAO1, the reaction will be added to a list with updated orthologous genes in PST.
#read the two library files for the lists, as well as the reaction list to call reactions
orthologs = pd.read_csv('../data/previous_data_from_curation/PA01_PSY_orthologs.csv')

#first create dictionary with the orthologous pairs
ortholog_dict = {}

for x in range(0, len(orthologs)):
    pair_row = orthologs.iloc[x,]
    query = pair_row['Locus Tag (Query)']
    result = pair_row['Locus Tag (Hit)']
    ortholog_dict[query]=result

    
# Copy the PAO1 model, since we will be "using it for parts" during curation.
pa_original = pa.copy()

# Now see if each reaction from PAO1 has a satisfied GPR based on
# The orthology. Reactions in PAO1 are used for GPR manipulation and
# then are added directly to our model.
unmapped_rxns = []
mapped_rxns = {}
for reaction in pa.reactions:
    rxn_gpr = reaction.gene_reaction_rule
    new_gpr = rxn_gpr
    # Search all of the orthologs against the GPR and replace any hits
    found_gene = False
    for gene in ortholog_dict.keys():
        if rxn_gpr.find(gene) > -1:
            # replace the ortholog in the new GPR
            new_gpr = new_gpr.replace(gene, ortholog_dict[gene])
            found_gene = True
            if reaction.id in mapped_rxns.keys():
                mapped_rxns[reaction.id].append(gene)
            else:
                mapped_rxns[reaction.id] = [gene]
    # If an ortholog was found for any genes in this GPR, see if
    # the GPR is functional when we swap out orthologs and 
    # remove remaining PAO1 genes.
    if found_gene:

        # convert to the AST structure to perform knockouts on the GPR
        as_tree, gene_set = parse_gpr(new_gpr)
        # knock out any PAO1 genes in the new GPR and check for functionality
        genes_to_ko = []
        for gene in reaction.genes:
            if gene.id.startswith('PA'):
                genes_to_ko.append(gene)
        ko_result = eval_gpr(as_tree,set([g.id for g in genes_to_ko]))
        # if we were able to knock out all the PAO1 genes and the reaction
        # was still functional, then we should remove the PAO1 genes from
        # the model and add any PST genes that were in functional reactions.
        if ko_result:
            reaction.gene_reaction_rule = new_gpr
            # remove the gene; don't remove the reaction yet, since we are 
            # currently iterating over the reactions (removal will cause
            # a skip for another reaction)
            cobra.manipulation.remove_genes(pa,genes_to_ko,remove_reactions=False)
        else: # if the knockout failed, the reaction doesn't fully map
            unmapped_rxns.append(reaction.id)
            
    else:
        # if the GPR is the same, the reaction does not map to PST
        # and we can remove it from the PAO1 model to simplify
        unmapped_rxns.append(reaction.id)
        
pa.remove_reactions(unmapped_rxns)

# Add the reactions remaining in PAO1. If they already exist in PST,
# just use the existing GPR. Report conflicts too (e.g., existing
# reaction that already has a GPR which differs from the orthology-
# based GPR).
rxns_to_add = []
conflicting_gprs = {}
for reaction in pa.reactions:
    if reaction.id+'_c' not in [r.id for r in model.reactions]:
        to_add = reaction.copy()
        to_add.id = to_add.id + '_c'
        rxns_to_add.append(to_add)
    else:
        # We will keep the annotated GPRs in modelSEED for now, but save the information.
        conflicting_gprs[reaction.id] = {'orthology-transferred GPR':reaction.gene_reaction_rule,
                                        'pst-annotation GPR in SEED':model.reactions.get_by_id(reaction.id+'_c').gene_reaction_rule}
        
pd.DataFrame(conflicting_gprs).T.to_csv('../results/future_curation/potential_GPR_conflicts.tsv',sep='\t')
model.add_reactions(rxns_to_add)

save_dir_model = filepath_model+filevar+filesuff_xml
write_dir = filepath_model + filevar + filesuff_csv

with open(write_dir, 'w', newline='') as csvfile:
    csvfile = csv.writer(csvfile, delimiter=',')
    for rxn in model.reactions:
        csvfile.writerow([rxn.id, rxn.name, rxn.reaction, rxn.gene_reaction_rule,rxn.lower_bound, rxn.upper_bound])

cobra.io.write_sbml_model(model, save_dir_model)



cobra/core/group.py:109 UserWarning: need to pass in a list


In [6]:
filevar = "PSTv2"
#biomass reactions are temporarily added for gapfilling purposes
bio1 = seed_draft.reactions.bio1.copy()
bio2 = pa_original.reactions.PAO1_Biomass.copy()
bio3 = pa_original.reactions.PA_Biomass_v13ub.copy()
bio4 = pa_original.reactions.PA_Biomass_v13.copy()
bio5 = pa_original.reactions.PA_Biomass_v4.copy()
model.add_reactions([bio1, bio2, bio3, bio4, bio5])


save_dir_model = filepath_model+filevar+filesuff_xml
write_dir = filepath_model + filevar + filesuff_csv

with open(write_dir, 'w', newline='') as csvfile:
    csvfile = csv.writer(csvfile, delimiter=',')
    for rxn in model.reactions:
        csvfile.writerow([rxn.id, rxn.name, rxn.reaction, rxn.gene_reaction_rule,rxn.lower_bound, rxn.upper_bound])

cobra.io.write_sbml_model(model, save_dir_model)

To get from v2 to v3 is the next step. This is the point where we added a lot of nomenclature for satisfying MEMOTE, including the inchie strings and other database knowledge. We are now integrating these steps after the rest of the curation, but are skipping to generation of v4 to keep our naming consistent with the previous approach.

In [7]:
filevar = "PSTv4_post_ensemble1"
  
#for specific information of the inclusion of reactions, including particularites of the evidence, please reference curate_from_ensemble_round1.ipynb        
##the following is from the first round of curation that took place, from the original gapfilling through ensemble number 1
rxn1 = universal.reactions.get_by_id('rxn02276_c').copy()
rxn1.gene_reaction_rule  = ' PSPTO_3554 '
rxn1.notes = {'ensemble_curation_step':1}
rxn2 = universal.reactions.get_by_id('rxn09072_c').copy()
rxn2.gene_reaction_rule = ' PSPTO_4214 '
rxn2.notes = {'ensemble_curation_step':1}
rxn3 = universal.reactions.get_by_id('rxn00993_c').copy()
rxn3.gene_reaction_rule = ' PSPTO_5292 '
rxn3.notes = {'ensemble_curation_step':1}
rxn4 = universal.reactions.get_by_id('rxn10878_c').copy()
rxn4.gene_reaction_rule = ' PSPTO_2510 '
rxn4.notes = {'ensemble_curation_step':1}
rxn5 = universal.reactions.get_by_id('rxn01418_c').copy()
rxn5.gene_reaction_rule = ' (PSPTO_2199 and PSPTO_2200 and PSPTO_2201) '
rxn5.notes = {'ensemble_curation_step':1}
model.add_reactions([rxn1, rxn2, rxn3, rxn4, rxn5])

save_dir_model = filepath_model+filevar+filesuff_xml
write_dir = filepath_model + filevar + filesuff_csv

with open(write_dir, 'w', newline='') as csvfile:
    csvfile = csv.writer(csvfile, delimiter=',')
    for rxn in model.reactions:
        csvfile.writerow([rxn.id, rxn.name, rxn.reaction, rxn.gene_reaction_rule,rxn.lower_bound, rxn.upper_bound])

cobra.io.write_sbml_model(model, save_dir_model)


#now create the ensemble, round two after the first set of curation was added to the base model
filevar = "PSTv4_post_ensemble2"

##the following is from the second round of ensemble gapfilling
rxn1 = universal.reactions.get_by_id('rxn01423_c').copy()
rxn1.gene_reaction_rule  = 'PSPTO_4775 or PSPTO_5504 or PSPTO_0096'
rxn1.notes = {'ensemble_curation_step':2}
rxn2 = universal.reactions.get_by_id('rxn03005_c').copy()
rxn2.gene_reaction_rule  = 'PSPTO_1468 or PSPTO_1699'
rxn2.notes = {'ensemble_curation_step':2}
rxn3 = universal.reactions.get_by_id('rxn00688_c').copy()
rxn3.gene_reaction_rule  = 'PSPTO_1468 or PSPTO_1699 or PSPTO_4866 or PSPTO_0178'
rxn3.notes = {'ensemble_curation_step':2}
rxn4 = universal.reactions.get_by_id('rxn01211_c').copy()
rxn4.gene_reaction_rule  = 'PSPTO_2453'
rxn4.notes = {'ensemble_curation_step':2}
rxn5 = universal.reactions.get_by_id('rxn00835_c').copy()
rxn5.gene_reaction_rule  = 'PSPTO_2022 and PSPTO_1131'
rxn5.notes = {'ensemble_curation_step':2}
rxn6 = universal.reactions.get_by_id('rxn05316_c').copy()
rxn6.gene_reaction_rule  = 'Unknown'
rxn6.notes = {'ensemble_curation_step':2}
rxn7 = universal.reactions.get_by_id('rxn13150_c').copy()
rxn7.gene_reaction_rule  = 'PSPTO_2453 and PSPTO_3733 and PSPTO_5069'
rxn7.notes = {'ensemble_curation_step':2}
rxn8 = universal.reactions.get_by_id('rxn10036_c').copy()
rxn8.gene_reaction_rule  = 'PSPTO_3503 and PSPTO_5530'
rxn8.notes = {'ensemble_curation_step':2}

#these reactions also came from the second ensemble, but were specifically focusing on arginine metabolism
rxn9 = universal.reactions.get_by_id('rxn05303_c').copy()
rxn9.gene_reactions_rule = 'Unknown'
rxn9.notes = {'ensemble_curation_step':2}
rxn10 = universal.reactions.get_by_id('rxn00245_c').copy()
rxn10.gene_reaction_rule = '(PSPTO_2662)'
rxn10.notes = {'ensemble_curation_step' :2}
rxn11 = universal.reactions.get_by_id('rxn01934_c').copy()
rxn11.gene_reaction_rule = '(PSPTO_2424 or PSPTO_2434 or PSPTO_3287 or PSPTO_3460 or PSPTO_3920)'
rxn11.notes = {'ensemble_curation_step' :2}
rxn12 = universal.reactions.get_by_id('rxn01827_c').copy()
rxn12.gene_reaction_rule = '(PSPTO_2346)'
rxn12.notes = {'ensemble_curation_step' :2}
model.add_reactions([rxn1, rxn2, rxn3, rxn4, rxn5, rxn6, rxn7, rxn8, rxn9, rxn10, rxn11, rxn12])


save_dir_model = filepath_model+filevar+filesuff_xml
write_dir = filepath_model + filevar + filesuff_csv

with open(write_dir, 'w', newline='') as csvfile:
    csvfile = csv.writer(csvfile, delimiter=',')
    for rxn in model.reactions:
        csvfile.writerow([rxn.id, rxn.name, rxn.reaction, rxn.gene_reaction_rule,rxn.lower_bound, rxn.upper_bound])

cobra.io.write_sbml_model(model, save_dir_model)



Ignoring reaction 'rxn02276_c' since it already exists.


After PSTv4 with all annotations added, first round of ensemble gapfilling took place, where we created an ensemble of 500 members, gapfilled to the biolog growth data, and looked for the most disparate reaction additions in alternative gapfilling solutions for curation.
After the first round was complete, we ended up with model_post_ensemble_curation_round1.xml. This was followed with a second wave of ensemble curation, with changes made to the base model to fill in the curated reactions, and another 500 member ensemble was created. Again the most disperate reactions were found within the solutions, and these were targeted for curation.
The framework of execution can be in the file "gapfill_to_ensemble.ipynb", which takes any clear positive growth on biolog data (>0.3OD) and creates an exchange reaction for the substrate. Gapfilling solutions are generated based on this script, where the number of ensemble members can be changed to generate ensembles of different sizes. The framework presented in the example script was used for each ensemble curation step. The next script, ensemble_learning.ipynb is to generate the list of the most disperate reactions, and finally, the output is an excel file with all of the reactions in their disperate form. From this, the script curate_from_ensemble***.ipynb is used as a journal for reaction additions or deletions to the model from the base model of the ensemble.
There is a drastic reduction in the number of metabolites and the number of reactions due to cleaning up exchange reactions and the previous biomass reactions as well.
The exchange reactions were introduced from the information in the Ma and Mhadmi papers, respectivly https://doi.org/10.1073/pnas.1319485111 and https://doi.org/10.1016/j.envexpbot.2014.07.002

From PSTv5 to PSTv6, the main changes include integrating exchange reactions from the Anderson 2014 paper, which looked at the differences in apoplasmic content of arabidopsis in normal and mkp1 mutants. This information is later used as in planta simulations for ensemble gene deletions. This is done in the notebook new_exchange_rxns2.ipynb.

In [8]:
#addition of the LPS pathway following, for more information on the inclusion of this pathway, please reference lps_curation.ipynb
filevar = 'PSTv5'

#lpxa = universal.reactions.rxn06729_c.copy()
#lpxa.gene_reaciton_rule = ("PSPTO_1546")
lpxc = universal.reactions.rxn03146_c.copy()
lpxc.gene_reaction_rule = ("PSPTO_4402")
#lpxd = universal.reactions.rxn06723_c.copy()
#lpxd.gene_reaction_rule = ("PSPTO_1542")
lpxh = universal.reactions.rxn03130_c.copy()
lpxh.gene_reaction_rule = ("PSPTO_3745")
lpxb1 = universal.reactions.rxn03159_c.copy()
lpxb1.gene_reaction_rule = ("PSPTO_1547")
lpxb2 = universal.reactions.rxn03181_c.copy()
lpxb2.gene_reaction_rule = ("PSPTO_1547")
kdta1 = universal.reactions.rxn03182_c.copy()
kdta1.gene_reaction_rule = ("PSPTO_4978")
kdta2 = universal.reactions.rxn03439_c.copy()
kdta2.gene_reaction_rule = ("PSPTO_4978")
htrb = universal.reactions.rxn06848_c.copy()
htrb.gene_reaction_rule = ("PSPTO_3871")

model.add_reactions([ lpxc, lpxh,lpxb1,lpxb2,kdta1,kdta2,htrb])

save_dir_model = filepath_model+filevar+filesuff_xml
write_dir = filepath_model + filevar + filesuff_csv

with open(write_dir, 'w', newline='') as csvfile:
    csvfile = csv.writer(csvfile, delimiter=',')
    for rxn in model.reactions:
        csvfile.writerow([rxn.id, rxn.name, rxn.reaction, rxn.gene_reaction_rule,rxn.lower_bound, rxn.upper_bound])

cobra.io.write_sbml_model(model, save_dir_model)

PSTv6 integrated the exchange reactions in an improper form, and this rendered them to not be marked as exchange reactions. To fix this, I added _e to the metabolite identifiers at the end of the exchange reactions in the original script of new_exchange_rxns2.ipynb. Further additions to PSTv6 include the LPS synthesis pathway. The two notebooks necessary for these changes are new_exchange_rxns2.ipynb and lps_ciration.ipynb.

The next major addition was the coronatine synthesis pathway, which was performed in cor_addition.ipynb and now presented here. This went two ways: the first attempt was to add each of the reactions independantly that were laid out in https://doi.org/10.1073/pnas.95.26.15469 . This is a putative pathway, but the most worked out one to date. However due to issues of mass balance in ACP, I collapsed the pathway to more reflect the basic syntehsis posed in Pseudomonas syringae Phytotoxins: Mode of Action, Regulation, and Biosynthesis by Peptide and Polyketide Synthetases (no doi presented).

In [9]:
filevar = 'PSTv6'

cor1 = Metabolite(
    'cpd16565_c',
    formula='C18H24NO4',
    name='Coronatine',
    compartment='c')
cor2 = Metabolite(
    'cpd16565_e',
    formula='C18H24NO4',
    name='Coronatine',
    compartment='e')

cpd00211_c = universal.metabolites.cpd00211_c.copy()

rxn1 = Reaction('cor_synth')
rxn1.name = 'Coronatine Synthesis'
rxn1.lower_bound = 0
rxn1.upper_bound = 1000
rxn1.gene_reaction_rule = ('PSPTO_0259 or PSPTO_0301 and PSPTO_4709 and PSPTO_4680 or PSPTO_4685 and PSPTO_4681 or PSPTO_4683 and PSPTO_4682 and PSPTO_4690 and PSPTO_4686 and PSPTO_4687')
rxn1.add_metabolites({model.metabolites.cpd00322_c: -1,
                     model.metabolites.cpd00029_c : -3,
                     cpd00211_c : -1,
                     model.metabolites.cpd00020_c : -1,
                     model.metabolites.cpd00011_c : 3,
                     model.metabolites.cpd00067_c : 2,
                     model.metabolites.cpd00001_c : 3,
                     cor1 : 1})
rxn2 = Reaction('cor_trans')
rxn2.name = 'Coronatine Transport'
rxn2.lower_bound = 0
rxn2.upper_bound = 1000
rxn2.add_metabolites({cor1 : -1,
                      cor2 : 1})
rxn3 = Reaction('EX_cpd16565_e')
rxn3.name = 'Coronatine Exchange'
rxn3.lower_bound = 0
rxn3.upper_bound = 1000
rxn3.add_metabolites({cor2: -1})

model.add_reactions ([rxn1, rxn2,rxn3])

save_dir_model = filepath_model+filevar+filesuff_xml
write_dir = filepath_model + filevar + filesuff_csv

with open(write_dir, 'w', newline='') as csvfile:
    csvfile = csv.writer(csvfile, delimiter=',')
    for rxn in model.reactions:
        csvfile.writerow([rxn.id, rxn.name, rxn.reaction, rxn.gene_reaction_rule,rxn.lower_bound, rxn.upper_bound])

cobra.io.write_sbml_model(model, save_dir_model)

To PSTv6, the next major change was the addition of the PST specific biomass equation, generating PSTv7. This is done through the notebook concise_gapfill.ipynb, which is a part of the gapfilling protocol. So, even though it is embedded in the gapfilling protocol, this will not change the outcome bcause it is drawing from the same source everytime. There is no additional model based with the actual biomass equation, however, using the adapted version of the gapfilling protocol, I have generated PSTv7 with the new biomass equation.

In [10]:
filevar = "PSTv7"
df = pd.read_csv("../data/bio1_biomass_adapted.csv")
for metabolite in model.reactions.bio1.metabolites:
    model.reactions.bio1.subtract_metabolites({metabolite : model.reactions.bio1.get_coefficient(metabolite.id)})
for row in df.itertuples():
    try:
        model.metabolites.get_by_id(row.id)
    except:
        if row.id in universal.metabolites:
            met_to_add = universal.metabolites.get_by_id(row.id)
            model.add_metabolites([met_to_add])
        else:
            pass
        
for row in df.itertuples():
    model.reactions.bio1.add_metabolites({model.metabolites.get_by_id(row.id) : row.coefficient})

#remove the previous temporary biomass equations:
model.remove_reactions(['PAO1_Biomass', 'PA_Biomass_v13ub', 'PA_Biomass_v13', 'PA_Biomass_v4'], remove_orphans = True)

#write to a model and a cav file of the version
save_dir_model = filepath_model+filevar+filesuff_xml
write_dir = filepath_model + filevar + filesuff_csv

with open(write_dir, 'w', newline='') as csvfile:
    csvfile = csv.writer(csvfile, delimiter=',')
    for rxn in model.reactions:
        csvfile.writerow([rxn.id, rxn.name, rxn.reaction, rxn.gene_reaction_rule,rxn.lower_bound, rxn.upper_bound])

cobra.io.write_sbml_model(model, save_dir_model)


PSTv7 underwent another gapfilling protocol to have one more final curation step, in which all the added reactions were queried for potential additional gprs to add to the given reactions. This was done in the notebook gpr_filler.ipynb, aptly named.
The resulting base model is named PSTv8.

In [11]:
filevar = 'PSTv8'
with open("../results/ensembles/pto_ensemble_100_updated102519.pickle", 'rb') as infile:
    ensemble = load(infile)
    
#modify the gprs to include the manually annotated gene:reactions pairs from KEGG and BLASTing
missing_gprs_df = pd.read_excel('../data/gene_annotations/gpr_kegg_missing_reactions.xlsx', columns = ['rxn_id', 'gpr'])
rxn_gpr_dict = {}
for idx, row in missing_gprs_df.iterrows():
    rxn_gpr_dict[row[0] + '_c'] = row[1]
    

for rxn,gpr in rxn_gpr_dict.items():
    if rxn in str(ensemble.base_model.reactions):
        if "Unknown" in gpr:
            pass
        else: 
            rxn_to_add = ensemble.base_model.reactions.get_by_id(rxn).copy()
            rxn_to_add.gene_reaction_rule = gpr
            model.add_reaction(rxn_to_add)

gene_associations = pd.read_excel('../results/gene_associations_to_add.xlsx')
association_dict = {}
for x in gene_associations.itertuples():
    association_dict[x.rxn[0:10]] = x.gene
for key, val in association_dict.items():
    if key in ensemble.base_model.reactions:
        rxn_to_add = ensemble.base_model.reactions.get_by_id(key).copy()
        rxn_to_add.gene_reaction_rule = val
        model.add_reaction(rxn_to_add)
        
#add reactions that are positivly annotated in the draft, and already have a matching gene reaction rule that fits the reaction annotated in the draft recon
missing_rxns = pd.read_excel("../data/missing_reactions_draft_v_v8.xlsx")
gprs_to_add_to_v8 = {}
for row in missing_rxns.itertuples():
    if row.evidence == 1:
        gprs_to_add_to_v8[row.id] = row.gpr
for key,value in gprs_to_add_to_v8.items():
    try:
        rxn_to_add = universal.reactions.get_by_id(key).copy()
        rxn_to_add.gene_reaction_rule = value
        model.add_reactions([rxn_to_add])
    except:
        pass
    
save_dir_model = filepath_model+filevar+filesuff_xml
write_dir = filepath_model + filevar + filesuff_csv

with open(write_dir, 'w', newline='') as csvfile:
    csvfile = csv.writer(csvfile, delimiter=',')
    for rxn in model.reactions:
        csvfile.writerow([rxn.id, rxn.name, rxn.reaction, rxn.gene_reaction_rule,rxn.lower_bound, rxn.upper_bound])

# Some reactions from the universal seem to have a '<' or '>' character that libSBML does not like.
# rewrite those notes here so we can save as SBML.
for reaction in model.reactions:
    if 'gapfill_data' in reaction.notes.keys():
        reaction.notes['gapfill_data'] = 'modelseed gapfill with new PST annotation'

cobra.io.write_sbml_model(model, save_dir_model)

Ignoring reaction 'rxn00541_c' since it already exists.
Ignoring reaction 'rxn00726_c' since it already exists.
Ignoring reaction 'rxn00727_c' since it already exists.
Ignoring reaction 'rxn03393_c' since it already exists.
Ignoring reaction 'rxn05607_c' since it already exists.
Ignoring reaction 'rxn03005_c' since it already exists.
Ignoring reaction 'rxn05303_c' since it already exists.
Ignoring reaction 'rxn00506_c' since it already exists.
Ignoring reaction 'rxn01388_c' since it already exists.
Ignoring reaction 'rxn01418_c' since it already exists.
Ignoring reaction 'rxn01827_c' since it already exists.
Ignoring reaction 'rxn00101_c' since it already exists.
Ignoring reaction 'rxn01211_c' since it already exists.
Ignoring reaction 'rxn00623_c' since it already exists.
Ignoring reaction 'rxn05183_c' since it already exists.
Ignoring reaction 'rxn02186_c' since it already exists.
Ignoring reaction 'rxn05289_c' since it already exists.
Ignoring reaction 'rxn00328_c' since it already 

PSTv8 can now undergo a final gapfilling step using the ensemble method to simulated growth on media and other analysis presented in this paper. See final_ensemble_gapfill.ipynb for this step.